In [1]:
!pip install rasterio

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install tqdm==4.33.0

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install geopandas

You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import os
import requests, zipfile, io
import numpy as np # linear algebra
import pandas as pd

import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
import matplotlib.pyplot as plt        
%matplotlib inline
from matplotlib.pyplot import imshow
from scipy import stats
from rasterio.merge import merge
import geopandas as gpd
from rasterio.mask import mask
from rasterio.plot import adjust_band
from rasterio.plot import reshape_as_raster, reshape_as_image
from shapely.geometry import mapping
import sys
from tqdm import tqdm
tqdm.pandas()
import gc
import psutil

from queue import PriorityQueue

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [5]:
train = gpd.read_file('data/train/train.shp')
train.dropna(subset=['geometry'], inplace = True)
train.reset_index(drop=True, inplace= True)
test = gpd.read_file('data/test/test.shp')
sample_sub = pd.read_csv('data/sample_submission.csv')

train_fe = train.copy()
test_fe = test.copy()

In [6]:
train_fe.head()

,Field_Id,Area,Subregion,Crop_Id_Ne,geometry
0,1,0.915905,3,8,POLYGON ((2467881.175041331 -3352032.059296422...
1,2,2.064410,3,6,"POLYGON ((2468428.90623293 -3351993.17305059, ..."
2,3,1.080300,3,8,POLYGON ((2468333.745293362 -3352058.485056543...
3,4,1.316190,3,8,POLYGON ((2468607.857733211 -3352012.337531495...
4,7,5.529220,3,8,"POLYGON ((2468646.08520868 -3351706.201920062,..."


### Create stats csv files from raster images

In [7]:
def get_values(arr, n):
    """
    This function get an array and return and array of length n: if the initial length is smaller we set the original values in 
    the middle and we pad the rest with the mean value, else we take the n middle values from the original array if the length is bigger.
    
    arr: list or np array of dim 1
    Contains the original values
    
    n : int
    The length of the returned array
    """
    l = len(arr)
    if(l<n):
        arr_mean = np.mean(arr)
        result = [0] * n
        begin = int((n-l)/2)
        for i in range(begin):
            result[i] = arr_mean
        for i in range(begin, l + begin):
            result[i] = arr[i- begin]
        for i in range(l + begin, n):
            result[i] = arr_mean
#         print('l<n ', result)
    else:
        middle = int((l-n)/2)
        result = arr[middle: middle + n]
#         print('l>n ' , result)
        
    return result

In [8]:
def get_image_statistics(out_image, n=10):
    # calculate ndvi https://automating-gis-processes.github.io/CSC/notebooks/L5/raster-map-algebra.html
    red_all = out_image[2] ## contains zero padding pixels
    red = red_all[red_all != 0]
#     print('red ', red)
    red_values = get_values(red, n)
    
    green_all = out_image[1]
    green = green_all[green_all != 0]
    green_values = get_values(green, n)
        
    blue_all = out_image[0]
    blue = blue_all[blue_all != 0]
    blue_values = get_values(blue, n)
    
    nir_all = out_image[3]
    nir = nir_all[nir_all != 0]
    nir_values = get_values(nir, n)

    np.seterr(divide='ignore', invalid='ignore') ## ignore dividing by zero numpy
    # Convert to floats
    red_all = red_all.astype('float')
    nir_all = nir_all.astype('float')
    # Calculate NDVI using numpy arrays
    ndvi_all = (nir_all - red_all) / (nir_all + red_all)
    ndvi = ndvi_all[ndvi_all != 0]
    ndvi = ndvi[~np.isnan(ndvi)]
    ndvi_values = get_values(ndvi, n)
#     print(ndvi)
#     print(ndvi)
#     print(ndvi.shape)   
    return [np.mean(red), np.std(red), np.min(red), np.max(red), *red_values,
           np.mean(green), np.std(green), np.min(green), np.max(green), *green_values,
           np.mean(blue), np.std(blue), np.min(blue), np.max(blue), *blue_values,
           np.mean(nir), np.std(nir), np.min(nir), np.max(nir), *nir_values,
           np.mean(ndvi), np.std(ndvi), np.min(ndvi), np.max(ndvi), *ndvi_values]

In [15]:
def calculate_ndvi_and_save(date):
    
    ## read shapefiles
    train = gpd.read_file('data/train/train.shp')
    train.dropna(subset=['geometry'], inplace = True)
    train.reset_index(drop=True, inplace= True)
    test = gpd.read_file('data/test/test.shp')
    test.dropna(subset=['geometry'], inplace = True)
    test.reset_index(drop = True, inplace= True)
    
    ## Create the output dataframes
    
    columns = [     'red_mean'+date, 'red_sd'+date, 'red_min'+date, 'red_max'+date,
               'red_val1'+date, 'red_val2'+date, 'red_val3'+date, 'red_val4'+date, 'red_val5'+date, 'red_val6'+date,
               'red_val7'+date, 'red_val8'+date, 'red_val9'+date, 'red_val10'+date,
               'red_val11'+date, 'red_val12'+date, 'red_val13'+date, 'red_val14'+date, 'red_val15'+date, 'red_val16'+date,
               'red_val17'+date, 'red_val18'+date, 'red_val19'+date, 'red_val20'+date,
                    'green_mean'+date, 'green_sd'+date, 'green_min'+date, 'green_max'+date,
               'green_val1'+date, 'green_val2'+date, 'green_val3'+date, 'green_val4'+date, 'green_val5'+date, 'green_val6'+date,
               'green_val7'+date, 'green_val8'+date, 'green_val9'+date, 'green_val10'+date,
               'green_val11'+date, 'green_val12'+date, 'green_val13'+date, 'green_val14'+date, 'green_val15'+date, 'green_val16'+date,
               'green_val17'+date, 'green_val18'+date, 'green_val19'+date, 'green_val20'+date,
                    'blue_mean'+date, 'blue_sd'+date, 'blue_min'+date, 'blue_max'+date,
               'blue_val1'+date, 'blue_val2'+date, 'blue_val3'+date, 'blue_val4'+date, 'blue_val5'+date, 'blue_val6'+date,
               'blue_val7'+date, 'blue_val8'+date, 'blue_val9'+date, 'blue_val10'+date,
               'blue_val11'+date, 'blue_val12'+date, 'blue_val13'+date, 'blue_val14'+date, 'blue_val15'+date, 'blue_val16'+date,
               'blue_val17'+date, 'blue_val18'+date, 'blue_val19'+date, 'blue_val20'+date,
                    'nir_mean'+date, 'nir_sd'+date, 'nir_min'+date, 'nir_max'+date,
               'nir_val1'+date, 'nir_val2'+date, 'nir_val3'+date, 'nir_val4'+date, 'nir_val5'+date, 'nir_val6'+date,
               'nir_val7'+date, 'nir_val8'+date, 'nir_val9'+date, 'nir_val10'+date,
               'nir_val11'+date, 'nir_val12'+date, 'nir_val13'+date, 'nir_val14'+date, 'nir_val15'+date, 'nir_val16'+date,
               'nir_val17'+date, 'nir_val18'+date, 'nir_val19'+date, 'nir_val20'+date,
                    'ndvi_mean'+date, 'ndvi_sd'+date, 'ndvi_min'+date, 'ndvi_max'+date,
               'ndvi_val1'+date, 'ndvi_val2'+date, 'ndvi_val3'+date, 'ndvi_val4'+date, 'ndvi_val5'+date, 'ndvi_val6'+date,
               'ndvi_val7'+date, 'ndvi_val8'+date, 'ndvi_val9'+date, 'ndvi_val10'+date,
               'ndvi_val11'+date, 'ndvi_val12'+date, 'ndvi_val13'+date, 'ndvi_val14'+date, 'ndvi_val15'+date, 'ndvi_val16'+date,
               'ndvi_val17'+date, 'ndvi_val18'+date, 'ndvi_val19'+date, 'ndvi_val20'+date]
    
    
    output_train = pd.DataFrame(columns = columns)
    output_test = pd.DataFrame(columns = columns)
    
#     ## read raster image
    raster = rasterio.open(os.path.join('mosaics', r"Mosaic_"+ date+ ".tiff"), count=4)
    
    ## Change the crs
    train  = train.to_crs(raster.crs.data)
    test  = test.to_crs(raster.crs.data)
    
    #
    n= 20
    
    geoms_train = train.geometry.values
    for index, geom in enumerate(tqdm(geoms_train)):
        feature = [mapping(geom)] #geojson format

        # the mask function returns an array of the raster pixels within this feature
        out_image, out_transform = mask(raster, feature, crop=True)
        stat_list = get_image_statistics(out_image, n)
#         print(stat_list)
#         print(len(stat_list))
        output_train.loc[index] = stat_list
        
    geoms_test = test.geometry.values
    for index, geom in enumerate(tqdm(geoms_test)):
        feature = [mapping(geom)] #geojson format

        # the mask function returns an array of the raster pixels within this feature
        out_image, out_transform = mask(raster, feature, crop=True)
        stat_list = get_image_statistics(out_image, n)
        output_test.loc[index] = stat_list
    
    
    output_test['Field_Id'] = test['Field_Id']
    output_train['Field_Id'] = train['Field_Id']
    
    return output_train, output_test

In [16]:
date = '2017-01-01'
out_train, out_test = calculate_ndvi_and_save(date)
out_train.to_csv(os.path.join('stats/train', 'train_stats'+date+'.csv'), index=False)
out_test.to_csv(os.path.join('stats/test', 'test_stats'+date+'.csv'), index=False)

100%|██████████| 1074/1074 [00:04<00:00, 235.43it/s]


In [17]:
out_train.head()

,red_mean2017-01-01,red_sd2017-01-01,red_min2017-01-01,red_max2017-01-01,red_val12017-01-01,red_val22017-01-01,red_val32017-01-01,red_val42017-01-01,red_val52017-01-01,red_val62017-01-01,...,ndvi_val122017-01-01,ndvi_val132017-01-01,ndvi_val142017-01-01,ndvi_val152017-01-01,ndvi_val162017-01-01,ndvi_val172017-01-01,ndvi_val182017-01-01,ndvi_val192017-01-01,ndvi_val202017-01-01,Field_Id
0,736.656716,123.179344,606.0,1166.0,656.0,657.0,626.0,613.0,631.0,657.0,...,0.701825,0.725241,0.736278,0.740137,0.738685,0.708936,0.625488,0.478883,0.643744,1
1,718.962500,116.523704,575.0,1133.0,593.0,620.0,641.0,634.0,647.0,660.0,...,0.757829,0.754221,0.750053,0.720881,0.731262,0.725861,0.727740,0.707412,0.676799,2
2,982.405063,115.820411,771.0,1331.0,978.0,1078.0,1016.0,964.0,849.0,807.0,...,0.523073,0.640978,0.607477,0.573634,0.540264,0.536953,0.564512,0.509333,0.482273,3
3,627.524752,125.432053,507.0,1010.0,547.0,678.0,696.0,512.0,550.0,557.0,...,0.720147,0.649425,0.496496,0.572837,0.696715,0.725540,0.711105,0.682378,0.521048,4
4,679.436019,120.161402,558.0,1469.0,636.0,635.0,617.0,603.0,596.0,605.0,...,0.715492,0.718172,0.710811,0.623615,0.426760,0.677753,0.709859,0.712672,0.712633,7


In [18]:
date = '2017-01-31'
out_train, out_test = calculate_ndvi_and_save(date)
out_train.to_csv(os.path.join('stats/train', 'train_stats'+date+'.csv'), index=False)
out_test.to_csv(os.path.join('stats/test', 'test_stats'+date+'.csv'), index=False)

100%|██████████| 1074/1074 [00:04<00:00, 227.07it/s]


In [19]:
date = '2017-02-10'
out_train, out_test = calculate_ndvi_and_save(date)
out_train.to_csv(os.path.join('stats/train', 'train_stats'+date+'.csv'), index=False)
out_test.to_csv(os.path.join('stats/test', 'test_stats'+date+'.csv'), index=False)

100%|██████████| 1074/1074 [00:04<00:00, 231.31it/s]


In [20]:
out_train.head()

,red_mean2017-02-10,red_sd2017-02-10,red_min2017-02-10,red_max2017-02-10,red_val12017-02-10,red_val22017-02-10,red_val32017-02-10,red_val42017-02-10,red_val52017-02-10,red_val62017-02-10,...,ndvi_val122017-02-10,ndvi_val132017-02-10,ndvi_val142017-02-10,ndvi_val152017-02-10,ndvi_val162017-02-10,ndvi_val172017-02-10,ndvi_val182017-02-10,ndvi_val192017-02-10,ndvi_val202017-02-10,Field_Id
0,676.985075,65.874030,585.0,850.0,646.0,621.0,585.0,593.0,605.0,622.0,...,0.762456,0.775897,0.776157,0.784056,0.779053,0.768607,0.714501,0.659929,0.714056,1
1,573.881250,104.541641,485.0,1026.0,555.0,499.0,496.0,491.0,489.0,499.0,...,0.802512,0.833648,0.833220,0.832660,0.833924,0.834889,0.832424,0.835048,0.819364,2
2,723.227848,50.097108,628.0,863.0,717.0,796.0,721.0,646.0,682.0,674.0,...,0.681818,0.701639,0.688606,0.684427,0.673759,0.678335,0.701438,0.679513,0.650809,3
3,607.762376,99.922460,513.0,986.0,539.0,654.0,595.0,527.0,555.0,572.0,...,0.764806,0.725838,0.614709,0.711237,0.774457,0.790743,0.777732,0.746113,0.653144,4
4,564.514218,99.985134,498.0,1457.0,528.0,522.0,521.0,526.0,550.0,579.0,...,0.819980,0.814570,0.813899,0.812752,0.592299,0.795669,0.816950,0.811802,0.813217,7


In [21]:
date = "2017-03-12"
out_train, out_test = calculate_ndvi_and_save(date)
out_train.to_csv(os.path.join('stats/train', 'train_stats'+date+'.csv'), index=False)
out_test.to_csv(os.path.join('stats/test', 'test_stats'+date+'.csv'), index=False)
out_test.head()

100%|██████████| 1074/1074 [00:04<00:00, 243.01it/s]


,red_mean2017-03-12,red_sd2017-03-12,red_min2017-03-12,red_max2017-03-12,red_val12017-03-12,red_val22017-03-12,red_val32017-03-12,red_val42017-03-12,red_val52017-03-12,red_val62017-03-12,...,ndvi_val122017-03-12,ndvi_val132017-03-12,ndvi_val142017-03-12,ndvi_val152017-03-12,ndvi_val162017-03-12,ndvi_val172017-03-12,ndvi_val182017-03-12,ndvi_val192017-03-12,ndvi_val202017-03-12,Field_Id
0,524.942857,130.641409,418.0,1129.0,489.0,454.0,445.0,462.0,588.0,617.0,...,0.768697,0.777015,0.779889,0.536150,0.713324,0.748490,0.758200,0.766144,0.775537,5
1,452.521127,58.839926,402.0,726.0,419.0,419.0,417.0,458.0,439.0,412.0,...,0.749591,0.772003,0.768511,0.760649,0.766309,0.766809,0.749045,0.750214,0.763143,6
2,699.726136,111.736393,506.0,1180.0,795.0,790.0,704.0,613.0,639.0,659.0,...,0.713515,0.698282,0.681727,0.672764,0.670594,0.666837,0.653620,0.625289,0.595244,10
3,722.946602,189.282760,523.0,1647.0,1030.0,658.0,611.0,609.0,616.0,643.0,...,0.667626,0.671958,0.667198,0.641108,0.606077,0.506629,0.616708,0.654858,0.662696,18
4,576.996979,67.925563,497.0,879.0,523.0,541.0,552.0,545.0,791.0,611.0,...,0.642769,0.671225,0.667175,0.669868,0.688289,0.690575,0.686114,0.681818,0.686397,23


In [22]:
date = "2017-03-22"
out_train, out_test = calculate_ndvi_and_save(date)
out_train.to_csv(os.path.join('stats/train', 'train_stats'+date+'.csv'), index=False)
out_test.to_csv(os.path.join('stats/test', 'test_stats'+date+'.csv'), index=False)
out_test.head()

100%|██████████| 1074/1074 [00:04<00:00, 244.52it/s]


,red_mean2017-03-22,red_sd2017-03-22,red_min2017-03-22,red_max2017-03-22,red_val12017-03-22,red_val22017-03-22,red_val32017-03-22,red_val42017-03-22,red_val52017-03-22,red_val62017-03-22,...,ndvi_val122017-03-22,ndvi_val132017-03-22,ndvi_val142017-03-22,ndvi_val152017-03-22,ndvi_val162017-03-22,ndvi_val172017-03-22,ndvi_val182017-03-22,ndvi_val192017-03-22,ndvi_val202017-03-22,Field_Id
0,702.676190,104.246457,590.0,1082.0,662.0,637.0,642.0,666.0,684.0,652.0,...,0.304716,0.306794,0.297267,0.155783,0.312651,0.265946,0.283288,0.297439,0.287344,5
1,611.422535,69.459414,543.0,890.0,586.0,573.0,553.0,773.0,580.0,555.0,...,0.295294,0.334545,0.303123,0.290479,0.318182,0.304914,0.242424,0.308378,0.305521,6
2,725.689773,110.250852,517.0,1221.0,836.0,804.0,741.0,653.0,670.0,678.0,...,0.298716,0.280837,0.278284,0.285175,0.291757,0.289659,0.280462,0.235056,0.213974,10
3,742.635922,205.742200,570.0,1952.0,1177.0,664.0,642.0,649.0,656.0,688.0,...,0.637207,0.632842,0.637320,0.626536,0.585848,0.508290,0.590983,0.639545,0.621135,18
4,592.646526,62.818677,518.0,893.0,547.0,544.0,559.0,579.0,787.0,574.0,...,0.617335,0.640485,0.641353,0.644741,0.660985,0.661260,0.659443,0.658125,0.652467,23


In [23]:
date = "2017-05-31"
out_train, out_test = calculate_ndvi_and_save(date)
out_train.to_csv(os.path.join('stats/train', 'train_stats'+date+'.csv'), index=False)
out_test.to_csv(os.path.join('stats/test', 'test_stats'+date+'.csv'), index=False)
out_test.head()

100%|██████████| 1074/1074 [00:04<00:00, 241.94it/s]


,red_mean2017-05-31,red_sd2017-05-31,red_min2017-05-31,red_max2017-05-31,red_val12017-05-31,red_val22017-05-31,red_val32017-05-31,red_val42017-05-31,red_val52017-05-31,red_val62017-05-31,...,ndvi_val122017-05-31,ndvi_val132017-05-31,ndvi_val142017-05-31,ndvi_val152017-05-31,ndvi_val162017-05-31,ndvi_val172017-05-31,ndvi_val182017-05-31,ndvi_val192017-05-31,ndvi_val202017-05-31,Field_Id
0,963.028571,58.922137,822.0,1161.0,966.0,940.0,917.0,989.0,933.0,822.0,...,0.415514,0.432480,0.459653,0.452615,0.407831,0.400726,0.429354,0.438087,0.423926,5
1,977.028169,83.979034,763.0,1122.0,955.0,987.0,1007.0,1043.0,922.0,923.0,...,0.468235,0.450146,0.442130,0.403468,0.361830,0.352544,0.357328,0.424955,0.409408,6
2,1078.269318,113.298235,699.0,1393.0,809.0,901.0,992.0,1139.0,1185.0,1172.0,...,0.332587,0.335265,0.342607,0.360511,0.358366,0.361650,0.367543,0.385440,0.431975,10
3,1274.058252,159.426084,1012.0,2107.0,1521.0,1340.0,1389.0,1381.0,1369.0,1323.0,...,0.271795,0.254994,0.246335,0.257192,0.244869,0.213018,0.296610,0.296626,0.285668,18
4,1197.172205,55.042120,1047.0,1355.0,1163.0,1159.0,1159.0,1221.0,1235.0,1292.0,...,0.261364,0.285550,0.292370,0.293843,0.304273,0.295783,0.309187,0.316405,0.296772,23


In [24]:
date = "2017-06-20"
out_train, out_test = calculate_ndvi_and_save(date)
out_train.to_csv(os.path.join('stats/train', 'train_stats'+date+'.csv'), index=False)
out_test.to_csv(os.path.join('stats/test', 'test_stats'+date+'.csv'), index=False)
out_test.head()

100%|██████████| 1074/1074 [00:04<00:00, 244.17it/s]


,red_mean2017-06-20,red_sd2017-06-20,red_min2017-06-20,red_max2017-06-20,red_val12017-06-20,red_val22017-06-20,red_val32017-06-20,red_val42017-06-20,red_val52017-06-20,red_val62017-06-20,...,ndvi_val122017-06-20,ndvi_val132017-06-20,ndvi_val142017-06-20,ndvi_val152017-06-20,ndvi_val162017-06-20,ndvi_val172017-06-20,ndvi_val182017-06-20,ndvi_val192017-06-20,ndvi_val202017-06-20,Field_Id
0,1003.104762,84.746223,869.0,1343.0,1033.0,994.0,940.0,919.0,942.0,977.0,...,0.356589,0.358306,0.374069,0.392027,0.350148,0.350543,0.361073,0.384251,0.368634,5
1,1040.394366,94.660621,841.0,1344.0,985.0,1034.0,1019.0,1051.0,1046.0,970.0,...,0.387436,0.398847,0.402450,0.380110,0.339347,0.323732,0.349112,0.361188,0.359878,6
2,1169.810227,106.475783,763.0,1470.0,921.0,998.0,1210.0,1179.0,1194.0,1198.0,...,0.291619,0.287175,0.300507,0.289529,0.269994,0.289786,0.302957,0.333533,0.393500,10
3,1339.456311,186.995186,1053.0,2443.0,1624.0,1509.0,1413.0,1411.0,1408.0,1389.0,...,0.238290,0.238462,0.219441,0.218437,0.230468,0.219028,0.230028,0.254890,0.254760,18
4,1314.012085,61.340020,1163.0,1484.0,1273.0,1246.0,1287.0,1296.0,1411.0,1287.0,...,0.182624,0.203243,0.234672,0.221982,0.231280,0.250381,0.257650,0.264134,0.245484,23


In [25]:
date = "2017-07-10"
out_train, out_test = calculate_ndvi_and_save(date)
out_train.to_csv(os.path.join('stats/train', 'train_stats'+date+'.csv'), index=False)
out_test.to_csv(os.path.join('stats/test', 'test_stats'+date+'.csv'), index=False)
out_test.head()

100%|██████████| 1074/1074 [00:04<00:00, 243.41it/s]


,red_mean2017-07-10,red_sd2017-07-10,red_min2017-07-10,red_max2017-07-10,red_val12017-07-10,red_val22017-07-10,red_val32017-07-10,red_val42017-07-10,red_val52017-07-10,red_val62017-07-10,...,ndvi_val122017-07-10,ndvi_val132017-07-10,ndvi_val142017-07-10,ndvi_val152017-07-10,ndvi_val162017-07-10,ndvi_val172017-07-10,ndvi_val182017-07-10,ndvi_val192017-07-10,ndvi_val202017-07-10,Field_Id
0,1075.800000,91.739735,902.0,1414.0,1092.0,1056.0,983.0,1012.0,1110.0,1150.0,...,0.307791,0.332107,0.319464,0.344508,0.329726,0.302972,0.330500,0.356374,0.337650,5
1,1108.253521,87.050692,905.0,1342.0,1064.0,1084.0,1064.0,1056.0,1142.0,1055.0,...,0.315284,0.366667,0.359928,0.323502,0.286292,0.282002,0.329612,0.316033,0.310387,6
2,1145.336364,96.560252,837.0,1401.0,984.0,1022.0,1192.0,1176.0,1202.0,1216.0,...,0.261822,0.268215,0.291534,0.278501,0.258463,0.259027,0.291012,0.329231,0.386091,10
3,1309.834951,155.989706,1031.0,2069.0,1581.0,1411.0,1376.0,1358.0,1343.0,1325.0,...,0.231796,0.223942,0.219103,0.226065,0.221625,0.196500,0.232103,0.240769,0.247023,18
4,1309.800604,69.587495,1150.0,1551.0,1310.0,1275.0,1312.0,1286.0,1390.0,1299.0,...,0.161345,0.203980,0.234630,0.214538,0.197858,0.215945,0.225437,0.226839,0.215541,23


In [26]:
date = "2017-07-15"
out_train, out_test = calculate_ndvi_and_save(date)
out_train.to_csv(os.path.join('stats/train', 'train_stats'+date+'.csv'), index=False)
out_test.to_csv(os.path.join('stats/test', 'test_stats'+date+'.csv'), index=False)
out_test.head()

100%|██████████| 1074/1074 [00:04<00:00, 237.82it/s]


,red_mean2017-07-15,red_sd2017-07-15,red_min2017-07-15,red_max2017-07-15,red_val12017-07-15,red_val22017-07-15,red_val32017-07-15,red_val42017-07-15,red_val52017-07-15,red_val62017-07-15,...,ndvi_val122017-07-15,ndvi_val132017-07-15,ndvi_val142017-07-15,ndvi_val152017-07-15,ndvi_val162017-07-15,ndvi_val172017-07-15,ndvi_val182017-07-15,ndvi_val192017-07-15,ndvi_val202017-07-15,Field_Id
0,1022.504762,100.703159,832.0,1354.0,1019.0,973.0,941.0,958.0,1027.0,1180.0,...,0.344622,0.357207,0.349357,0.364309,0.356127,0.337629,0.373941,0.388113,0.370084,5
1,1054.380282,93.059782,837.0,1308.0,1010.0,1037.0,1041.0,1038.0,1058.0,994.0,...,0.350966,0.380850,0.369972,0.343007,0.319987,0.320996,0.362903,0.349083,0.356831,6
2,1088.096591,106.445552,759.0,1466.0,872.0,922.0,1096.0,1064.0,1136.0,1148.0,...,0.332541,0.308316,0.332718,0.333943,0.298572,0.312094,0.349723,0.389982,0.442050,10
3,1266.432039,170.903461,941.0,2169.0,1683.0,1294.0,1308.0,1278.0,1237.0,1216.0,...,0.244546,0.248411,0.234018,0.255196,0.238285,0.227030,0.237943,0.278417,0.272035,18
4,1284.280967,82.869585,1093.0,1539.0,1293.0,1257.0,1270.0,1239.0,1362.0,1264.0,...,0.173289,0.219217,0.265064,0.239283,0.210295,0.224181,0.250872,0.251763,0.243645,23


In [27]:
date = "2017-08-04"
out_train, out_test = calculate_ndvi_and_save(date)
out_train.to_csv(os.path.join('stats/train', 'train_stats'+date+'.csv'), index=False)
out_test.to_csv(os.path.join('stats/test', 'test_stats'+date+'.csv'), index=False)
out_test.head()

100%|██████████| 1074/1074 [00:04<00:00, 240.47it/s]


,red_mean2017-08-04,red_sd2017-08-04,red_min2017-08-04,red_max2017-08-04,red_val12017-08-04,red_val22017-08-04,red_val32017-08-04,red_val42017-08-04,red_val52017-08-04,red_val62017-08-04,...,ndvi_val122017-08-04,ndvi_val132017-08-04,ndvi_val142017-08-04,ndvi_val152017-08-04,ndvi_val162017-08-04,ndvi_val172017-08-04,ndvi_val182017-08-04,ndvi_val192017-08-04,ndvi_val202017-08-04,Field_Id
0,1320.819048,196.337988,912.0,1691.0,1277.0,1291.0,1337.0,1391.0,1425.0,1482.0,...,0.169213,0.165882,0.172905,0.222001,0.186451,0.160593,0.181376,0.178327,0.149687,5
1,1317.154930,196.458048,1037.0,1606.0,1161.0,1100.0,1074.0,1051.0,1454.0,1364.0,...,0.168527,0.190260,0.233590,0.241255,0.250228,0.288875,0.297498,0.184277,0.209209,6
2,1049.605682,87.830222,828.0,1362.0,1088.0,1101.0,970.0,952.0,994.0,1022.0,...,0.277329,0.258316,0.289759,0.294280,0.267238,0.293532,0.309275,0.377572,0.457011,10
3,1365.961165,187.240639,1040.0,2365.0,1832.0,1522.0,1464.0,1422.0,1430.0,1382.0,...,0.218564,0.227862,0.227187,0.208412,0.204691,0.188008,0.209135,0.246141,0.256919,18
4,1415.220544,93.590367,1240.0,1738.0,1339.0,1340.0,1394.0,1349.0,1512.0,1364.0,...,0.160020,0.206501,0.255521,0.229489,0.206349,0.223130,0.221234,0.226421,0.224537,23


In [28]:
date = "2017-08-19"
out_train, out_test = calculate_ndvi_and_save(date)
out_train.to_csv(os.path.join('stats/train', 'train_stats'+date+'.csv'), index=False)
out_test.to_csv(os.path.join('stats/test', 'test_stats'+date+'.csv'), index=False)
out_test.head()

100%|██████████| 1074/1074 [00:04<00:00, 242.69it/s]


,red_mean2017-08-19,red_sd2017-08-19,red_min2017-08-19,red_max2017-08-19,red_val12017-08-19,red_val22017-08-19,red_val32017-08-19,red_val42017-08-19,red_val52017-08-19,red_val62017-08-19,...,ndvi_val122017-08-19,ndvi_val132017-08-19,ndvi_val142017-08-19,ndvi_val152017-08-19,ndvi_val162017-08-19,ndvi_val172017-08-19,ndvi_val182017-08-19,ndvi_val192017-08-19,ndvi_val202017-08-19,Field_Id
0,1460.723810,118.780991,1212.0,1794.0,1499.0,1448.0,1408.0,1390.0,1506.0,1687.0,...,0.254771,0.259094,0.259139,0.275862,0.258755,0.244924,0.257009,0.270404,0.264289,5
1,1485.985915,85.309791,1358.0,1744.0,1426.0,1436.0,1435.0,1399.0,1552.0,1451.0,...,0.250118,0.271930,0.276095,0.259727,0.236168,0.241813,0.270466,0.252029,0.266129,6
2,1352.794318,110.922730,1089.0,1672.0,1177.0,1218.0,1409.0,1359.0,1391.0,1372.0,...,0.260175,0.241796,0.275171,0.281527,0.262645,0.274573,0.297929,0.335245,0.399798,10
3,1543.427184,186.820047,1228.0,2517.0,2006.0,1583.0,1600.0,1545.0,1532.0,1527.0,...,0.263705,0.261575,0.253454,0.233981,0.222408,0.210818,0.248604,0.291473,0.298613,18
4,1594.604230,93.310014,1393.0,1917.0,1620.0,1576.0,1610.0,1564.0,1657.0,1593.0,...,0.171910,0.231088,0.269857,0.258275,0.221536,0.234970,0.235560,0.235999,0.241856,23


## based on k nearest neighbours

In this section we will calculate the occurance of each class near the polygon

In [45]:
def k_nearest_neighbor(r, k):
    q = PriorityQueue()
    centroid = r['centroid']
#     truth = r['Crop_Id_Ne']
    for index, row in train_fe.iterrows():
        crop_type = int(row['Crop_Id_Ne'])
        dist = centroid.distance(row['centroid'])
        if(dist != 0.0):
            q.put((dist,crop_type))
    
#     print('truth ', truth)
    occurances = [0]*9
    for i in range(k):
        tmp = q.get()
#         print(tmp)
        occurances[tmp[1]-1] +=1
        
#     print(occurances)
    return pd.Series(occurances)

In [52]:
train_fe['area'] = train_fe['geometry'].area
train_fe['centroid'] = train_fe['geometry'].centroid
test_fe['area'] = test_fe['geometry'].area
test_fe['centroid'] = test_fe['geometry'].centroid

In [53]:
k = 5
train_neighbors = train_fe.progress_apply(lambda row: k_nearest_neighbor(row, k), axis = 1)
test_neighbors = test_fe.progress_apply(lambda row: k_nearest_neighbor(row, k), axis = 1)

100%|██████████| 1074/1074 [06:20<00:00,  2.84it/s]


In [54]:
## Specify the columns names
columns = ['occ_neigh_1', 'occ_neigh_2', 'occ_neigh_3', 'occ_neigh_4', 'occ_neigh_5', 'occ_neigh_6', 'occ_neigh_7', 'occ_neigh_8', 'occ_neigh_9']
train_neighbors.columns = columns
test_neighbors.columns = columns


train_neighbors['Field_Id'] = train_fe['Field_Id']
test_neighbors['Field_Id'] = test_fe['Field_Id']

train_neighbors.head()

,occ_neigh_1,occ_neigh_2,occ_neigh_3,occ_neigh_4,occ_neigh_5,occ_neigh_6,occ_neigh_7,occ_neigh_8,occ_neigh_9,Field_Id
0,0,0,0,0,0,2,0,2,1,1
1,0,0,0,1,0,0,0,3,1,2
2,1,0,0,0,0,1,0,2,1,3
3,1,0,0,1,0,1,0,1,1,4
4,0,0,0,1,0,1,0,3,0,7


In [55]:
test_neighbors.head()

,occ_neigh_1,occ_neigh_2,occ_neigh_3,occ_neigh_4,occ_neigh_5,occ_neigh_6,occ_neigh_7,occ_neigh_8,occ_neigh_9,Field_Id
0,0,0,0,1,0,1,0,2,1,5
1,0,0,0,1,0,1,0,2,1,6
2,0,0,0,0,0,0,0,5,0,10
3,0,0,0,0,0,0,0,5,0,18
4,0,0,1,0,1,0,1,2,0,23


In [56]:
train_neighbors.to_csv(os.path.join('stats/train', 'train_neighbors5.csv'), index = False)
test_neighbors.to_csv(os.path.join('stats/test', 'test_neighbors5.csv'), index = False)